
---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

"## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [40]:
! cp readonly/test.csv   test.csv
! cp readonly/latlons.csv   latlons.csv
! cp readonly/addresses.csv   addresses.csv

In [41]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.svm import SVC
# import matplotlib.pyplot as plt

def blight_model():

    # Your code here
    train_data = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_data = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
#     address = pd.read_csv('addresses.csv')
#     latlon = pd.read_csv('latlons.csv')

#     join1 = pd.merge(address, latlon, how='inner', on='address').drop(['address'], axis = 1)
#     test_data = pd.merge(join1, test_data, how='inner', on='ticket_id')

    train_data = train_data[(train_data['compliance']==1) | (train_data['compliance']==0)]
#     address = pd.merge(address, latlon, how='inner', on='address').drop(['address'], axis=1)
#     train_data = pd.merge(train_data, address, how='inner', on='ticket_id')

    violation_code = train_data.groupby(train_data['violation_code']).agg({'compliance':['sum','count']})
#     violation_description = train_data.groupby(train_data['violation_description']).agg({'compliance':['sum','count']})
    vio_code_dict = dict(violation_code.iloc[:,0]/violation_code.iloc[:,1])
#     vio_des_dict = dict(violation_description.iloc[:,0]/violation_description.iloc[:,1])
    train_data = train_data.replace({'violation_code':vio_code_dict})
    test_data = test_data.replace({'violation_code':vio_code_dict})
    for i,row in test_data.iterrows():
        if type(row['violation_code'])!=float:
            test_data.loc[i,'violation_code']=np.nan
#         if type(row['violation_description'])!=float:
#             test_data.loc[i,'violation_description'] = np.nan
    test_data.fillna(0,inplace=True)

    leaky_columns = ['payment_amount',
                    'payment_date',
                    'payment_status',
                    'balance_due',
                    'collection_status',
                    'compliance_detail']
    waste = ['agency_name','inspector_name','violator_name','violation_zip_code','violation_street_name','violation_street_number','mailing_address_str_number',
             'violation_description','mailing_address_str_name','city','state','zip_code','non_us_str_code','country','ticket_issued_date','hearing_date','disposition','grafitti_status',
             'fine_amount','state_fee','admin_fee', 'clean_up_cost']

    X_train = train_data.drop(leaky_columns, axis=1).drop(waste, axis=1).dropna().set_index(['ticket_id'])
    Y_train = X_train['compliance']
    X_train = X_train.drop(['compliance'], axis=1)
    X_test = test_data.drop(waste, axis=1).set_index(['ticket_id']).fillna(method='ffill')
    ticket_id = X_test.index
#     x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, random_state=0)
    #     scaler = MinMaxScaler()
    #     X_train = scaler.fit_transform(X_train)
    #     X_test = scaler.transform(X_test)
    # grid_params = {'n_estimators':[50,100,150], 'max_depth':[3,7,10,15,20]}
    # grid_params = {'max_depth': [15,20,10,None]}
    # grid_params = {'n_neighbors':[3,10,15,30]}
    clf = RandomForestClassifier(n_estimators = 100, max_depth=30)

    # clf = LogisticRegression()
    # clf = KNeighborsClassifier(n_neighbors=30)
    # clf = SVC().fit(x_train, y_train)
    # clf = DecisionTreeClassifier() 
    clf.fit(X_train, Y_train)
    # clf = GridSearchCV(clf, param_grid=grid_params, scoring='roc_auc').fit(x_train, y_train)
    y_predict = pd.DataFrame(clf.predict_proba(X_test)[:,1], columns=['y_predict'])
    y_predict['ticket_id'] = ticket_id
    y_predict = y_predict.set_index(['ticket_id'])
    
    

#     print(clf.score(x_train, y_train), clf.score(x_test, y_test))#, clf.best_params_)
    return y_predict['y_predict']

In [42]:
blight_model()


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.189149
285362    0.014768
285361    0.064818
285338    0.047496
285346    0.083022
285345    0.047496
285347    0.070571
285342    0.390260
285530    0.014768
284989    0.016683
285344    0.052956
285343    0.014768
285340    0.014768
285341    0.052956
285349    0.083022
285348    0.047496
284991    0.016683
285532    0.016683
285406    0.016683
285001    0.014852
285006    0.000000
285405    0.014768
285337    0.016683
285496    0.052956
285497    0.047496
285378    0.014768
285589    0.016683
285585    0.047496
285501    0.064818
285581    0.014768
            ...   
376367    0.042205
376366    0.042692
376362    0.042692
376363    0.068835
376365    0.042205
376364    0.042692
376228    0.042692
376265    0.042692
376286    0.323416
376320    0.042692
376314    0.042692
376327    0.323416
376385    0.323416
376435    0.417548
376370    0.323416
376434    0.070571
376459    0.063409
376478    0.000000
376473    0.042692
376484    0.016256
376482    0.016701
37

In [43]:
# pd.Series(clf.predict(X_test)).value_counts()

In [44]:
# confusion_matrix(Y_train, clf.predict(X_train))


In [45]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.plot(fpr, tpr)
# plt.show()
# test_data['disposition'].value_counts()

In [46]:
# train_data['disposition'].value_counts()

In [47]:
# from adspy_shared_utilities import plot_class_regions_for_classifier
# plot_class_regions_for_classifier(clf,X_train, Y_train.astype(int))

In [48]:
# X_train